In [1]:
import requests
import re
import numpy as np
import json
import pandas as pd

In [2]:
corp = []

In [3]:
with open("gyakori_allatok_14000", "r+") as f:
    corp = json.loads(f.read())
    
with open("gyakori_szorakozas_30000", "r+") as f:
    corp += json.loads(f.read())

with open("gyakori_egeszseg", "r+") as f:
    corp += json.loads(f.read())
    
with open("gyakori_egeszseg_20000", "r+") as f:
    corp += json.loads(f.read())
    
with open("gyakori_szamitastechnika", "r+") as f:
    corp += json.loads(f.read())


In [4]:
corp[0]

{'hosszu_kerdes': 'Nagy terület őrzésére milyen kutyát javasolnátok? Bővebben lent. Régóta szerettünk volna kutyát tartani,de az igényeinknek nem tudjuk,hogy milyen kutya felelne meg,illetve,hogy mi milyen kutyának felelnénk meg.Ezért szeretném a segítségét kérni pár tapasztaltabb kutyásnak.Nem arról van szó,hogy nem tartottam még kutyát,hanem fajtákkal nem foglalkoztam túl sokat vagy épp nem én választottam ki azt.Párat én is ismerek,de nem annyira behatóan,hogy haza merjek hozni egyet mindenféle ismerettség nélkül.Adott egy nagy több hektáros terület,több féle-fajta-méretű állattal,így elsősorban egy állatokkal kompatibilis kutyára lenne szükségünk (lovak,egy szamár,szárnyasok,kecskék,pár bárány,struccok és nyulak).Vadász kutyákat ezért ki is lőttük,bár a következő pont miatt meg lehet,hogy pont ők felelnének meg.:Erdő és hegyek vannak mellettünk ahonnan vadállatok (főleg róka szokott) be tudnak jönni ezektől jó lenne,ha nem tartana.Nem kell,hogy öldököljön,csak azért ne hagyja őket 

In [5]:
df = pd.DataFrame(corp)

In [6]:
clean_questions = []
with open("clean_questions", "r") as f:
    for line in f:
        line = line.strip("\n")
        clean_questions.append(line)
        
df['hosszu_kerdes_clean'] = clean_questions

In [7]:
df["hosszu_kerdes_clean"]

0         nagy terület őrzés kutya javasol Bőveb lent ré...
1         az álllatorvos alig elér ez szerintetek normál...
2         Husky szoktat cica a cica először kutyus össze...
3         Jár kutya fej jobb balra szem perc mitől éves ...
4         Vadász vennék vizslát hobbi cél elvisz vadásza...
5         Elszállásolhatok kutya panellakás nap a testvé...
6         milyen tapasztalat vonat kutya utazás kapcsola...
7         Macskám betegség hát jön kiütés dudor Viszk pá...
8         Tudnát ajánl keverék kutyus elvan panel Sziasz...
9         Kutya csontszilánk eszik ma pörkölt ebéd én es...
10        mi csinál bánt Hozik kiscica hetes kislány hón...
11        a svájci fejár juhászkutya ajánlanát első kuty...
12        mi jelent fajtatiszta jellegű Sziaszt szeret k...
13        Sorrend állítanát alábbi fajta mozgásigény Csi...
14        Whippet kutya eltér valami van kutya whipp sze...
15        Mennyi idő mire macska megszokik otthon éves k...
16        Mekkora áramütés bír kutya a k

In [8]:
df["fokategoria"] = df.kategoriak.apply(lambda x: x[0])
df["alkategoria"] = df.kategoriak.apply(lambda x: x[1])
df = df.drop("kategoriak", axis=1)

In [9]:
import re
def filter_question(question):
    question = re.sub(r'(.+) nevű felhasználó kérdése:', '', question)
    return question

In [ ]:
df.hosszu_kerdes = df.hosszu_kerdes.apply(filter_question)